<a href="https://colab.research.google.com/github/urosgodnov/BigData/blob/master/textMining_scraping_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Permanently install libraries

## Initializing gdrive

In [1]:
# permanently install packageg in google colab
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Creating links

In [5]:
import os
nb_path = '/content/drive/MyDrive/Google_Colab_modules'

if not os.path.exists(nb_path):
  os.symlink('/content/drive/MyDrive/Google_Colab_modules', nb_path)  # Create the symlink only if it doesn't exist
else:
  print(f"Path '{nb_path}' already exists. Skipping symlink creation.")


Path '/content/drive/MyDrive/Google_Colab_modules' already exists. Skipping symlink creation.


## Installing to destination folder in gdrive

In [ ]:
#install the module in the
!pip install --target=$nb_path selenium

In [ ]:
!pip install --target=$nb_path webdriver_manager

## Connecting the path

In the future, while all modules will be installed, we will only use this command.

In [25]:
import sys

sys.path.append("/content/drive/MyDrive/Google_Colab_modules")

# Webscraping

In [194]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    ElementClickInterceptedException,
    TimeoutException
)
from datetime import datetime



In [195]:
import pandas as pd
import time

In [196]:
options = webdriver.ChromeOptions()
options.add_argument("--verbose")
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument("--window-size=1920, 1200")
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like \
Gecko) Chrome/90.0.4430.212 Safari/537.36')

In [197]:
driver = webdriver.Chrome(options=options)

In [198]:
l="https://www.imdb.com/title/tt23558280/reviews/?ref_=tt_urv_sm"
driver.get(l)

**Overlay page**

In [200]:
wait = WebDriverWait(driver, timeout=5)


In [ ]:
while True:
    try:

        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'ipc-see-more__button') and .//span[text()='All']]")))
        print('Found the "All" button.')

        # Scroll the button into view
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
        print('Scrolled to the "All" button.')

        # Click the button
        button.click()
        print('Button clicked successfully!')

    except TimeoutException as e:
        print('Timeout waiting for the button to be clickable:', e)
        break

    except ElementClickInterceptedException as e:
        print('Element click intercepted:', e)
        driver.execute_script("arguments[0].click();", button)
        print('Button clicked using JavaScript.')

    except NoSuchElementException:
        print('Button not found on the page.')
        break

    except Exception as e:
        print('An unexpected error occurred:', e)
        break


In [186]:
from bs4 import BeautifulSoup

In [187]:
df = pd.DataFrame(columns=["Author","Title","Rating","Text","Date"])



In [188]:
content = driver.page_source
soup = BeautifulSoup(content)
result=[]

In [189]:
def GetAReview(review):
  author = rev.findAll('a', attrs={'class':'ipc-link ipc-link--base'})[0].text
  title = rev.find('h3', attrs={'class':'ipc-title__text'}).text.strip()
  rating =rev.findAll('span', attrs={'class':'ipc-rating-star--rating'})[0].text.replace("\n","").strip()
  review = rev.findAll('div', attrs={'class':'ipc-html-content-inner-div'})[0].text
  date = rev.findAll('li', attrs={'class':'ipc-inline-list__item review-date'})[0].text
  date=datetime.strptime(date, '%b %d, %Y').strftime('%Y-%m-%d')

  return pd.DataFrame({"Author":author, "Title":title, "Rating":rating, "Text":review, "Date":date},index=[0])


In [190]:
for rev in soup.findAll('article'):
  try:
    result.append(GetAReview(rev))
  except:
    continue

In [191]:
df=pd.concat(result, axis=0, ignore_index=True)

In [192]:
df.head()

,Author,Title,Rating,Text,Date
0,hunterwinters-30598,"Interesting concept, so many questions",7,It was an interesting concept and an entertain...,2024-11-11
1,FeastMode,"Decent if it was a streaming movie, underwhelm...",5,I watched Elevation early as part of Marcus's ...,2024-11-07
2,Name657,Elevation,5,How do these low budget movie companies no one...,2024-11-10
3,jmc4769,A cheaper knock off of The Quiet Place but sti...,6,This movie is much better than the bad reviews...,2024-11-22
4,stofuia-67824,Good Movie worth the watch,7,Started as normal end of world storyline. Huma...,2024-11-08


In [ ]:
df.to_csv("Mavericks.csv")